File plan
- Format placement to: instream in-stream
infeed -> in-feed
newsfeed -> in-feed

- Unit -> buying_model

File actual:
Camp_type -> Objective


Cách match:
- Các campaign, có startdate và enddate nằm trong khoảng startdate và enddate của actual, is:
    - Category
    - Brand
    - Campaign
    - Funnel
    - Platform
    - Region
    - TA
    - Placement contain
    - Buying model
    - Ad format (trong file act sẽ contain VID, có s đằng sau number)

# 1. Import Functions

In [14]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Integer, Float, String, Text, Date, DateTime, Boolean, MetaData

def get_data(cursor, table_name, start_date=None, end_date=None):
    # If no dates provided, fetch all
    if not start_date and not end_date:
        query = f"SELECT * FROM {table_name}"
    else:
        # Default to 10-day window if only one date is missing
        if not end_date:
            end_date = datetime.now()
        if not start_date:
            start_date = end_date - timedelta(days=10)

        # Ensure strings
        if isinstance(start_date, datetime):
            start_str = start_date.strftime('%Y-%m-%d')
        else:
            start_str = str(start_date)

        if isinstance(end_date, datetime):
            end_str = end_date.strftime('%Y-%m-%d')
        else:
            end_str = str(end_date)

        query = f"""
            SELECT * FROM {table_name}
            WHERE report_date >= '{start_str}' AND report_date <= '{end_str}'
        """

    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    df = pd.DataFrame(rows, columns=column_names)
    return df

def split_and_assign(df, column, column_type):
    # Define expected columns inside the function
    expected_columns_map = {
        "campaign_name": ['camp_code', 'platform', 'camp_type', 'buying_type', 'free_text1', 'year', 'optimization', 'buying_model', 'placement', 'freetext_2'],
        "adgroup_name": ['noneed_1', 'noneed_2', 'noneed_3', 'noneed_4', 'noneed_5', 'noneed_6', 'noneed_7', 'noneed_8', 'bidding_strategy', 'noneed_9'],
        "ad_name": ['creative_type', 'creative_format', 'creative_length', 'noneed_9', 'noneed_10', 'noneed_11', 'noneed_12', 'noneed_13', 'freetext_3'],
    }

    expected_columns = expected_columns_map.get(column_type, [])

    if column in df.columns:
        df[column] = df[column].fillna("").astype(str)
        split_cols = df[column].str.split('_', expand=True)

        # Pad with NaNs if needed
        num_actual_cols = split_cols.shape[1]
        num_expected_cols = len(expected_columns)
        if num_actual_cols < num_expected_cols:
            for i in range(num_actual_cols, num_expected_cols):
                split_cols[i] = pd.NA
        num_actual_cols = split_cols.shape[1]

        assigned_names = expected_columns[:num_actual_cols]
        extra_names = [f"extra_{i}" for i in range(num_actual_cols - len(assigned_names))]
        all_new_names = assigned_names + extra_names
        all_new_names = [f"{name}_split" if name in df.columns else name for name in all_new_names]

        split_cols.columns = all_new_names
        df = df.join(split_cols)

    # Split 'camp_code' into components
    if "camp_code" in df.columns:
        df["camp_code"] = df["camp_code"].fillna("").astype(str)
        campaign_split = df["camp_code"].str.split("-", expand=True)

        max_cols = 6
        campaign_split = campaign_split.iloc[:, :max_cols]
        for i in range(campaign_split.shape[1], max_cols):
            campaign_split[i] = np.nan

        campaign_split.columns = ["Funnel", "Category", "Brand Name", "Campaign Name", "Start Date", "End Date"]
        df = df.drop(columns=[col for col in campaign_split.columns if col in df.columns], errors="ignore")
        df = df.join(campaign_split)

    # Split 'free_text1' into components
    if "free_text1" in df.columns:
        df["free_text1"] = df["free_text1"].fillna("").astype(str)
        freetext_split = df["free_text1"].str.split("-", expand=True)

        max_cols = 6
        freetext_split = freetext_split.iloc[:, :max_cols]
        for i in range(freetext_split.shape[1], max_cols):
            freetext_split[i] = np.nan

        freetext_split.columns = ["Audience", "Gender", "Age", "Region", "Device", "Os"]
        df = df.drop(columns=[col for col in freetext_split.columns if col in df.columns], errors="ignore")
        df = df.join(freetext_split)

    # Create TA column
    if "Gender" in df.columns and "Age" in df.columns:
        df["Gender"] = df["Gender"].astype(str)
        df["Age"] = df["Age"].astype(str).str.zfill(4)
        df["TA"] = df["Gender"] + df["Age"].str[:2] + "-" + df["Age"].str[2:]

    # Format Start and End Dates
    if "Start Date" in df.columns and "End Date" in df.columns and "year" in df.columns:
        df["year"] = df["year"].astype(str)

        def format_date(row, col):
            if pd.notna(row[col]) and len(row[col]) == 4:
                return f"{row['year'][:2]}{row[col][:2]}-{row[col][2:]}"
            return None

        df["Start Date"] = df.apply(lambda row: format_date(row, "Start Date"), axis=1)
        df["End Date"] = df.apply(lambda row: format_date(row, "End Date"), axis=1)

        df["Start Date"] = pd.to_datetime(df["Start Date"], format="%Y-%m-%d", errors="coerce")
        df["End Date"] = pd.to_datetime(df["End Date"], format="%Y-%m-%d", errors="coerce")

    # Create ad_format column
    if all(col in df.columns for col in ["creative_type", "creative_format", "creative_length"]):
        df["ad_format"] = df["creative_type"].astype(str) + "_" + df["creative_format"].astype(str) + "_" + df["creative_length"].astype(str)

    # Drop unnecessary columns
    df = df.drop(columns=[col for col in df.columns if "noneed" in col], errors="ignore")

    return df


def upload_to_mysql(df, table_name, engine, if_exists='append'):
    """
    Uploads a DataFrame to a MySQL table with correct column types based on the DataFrame.

    :param df: Pandas DataFrame to upload
    :param table_name: Name of the table in the database
    :param engine: SQLAlchemy engine object
    :param if_exists: What to do if the table already exists ('fail', 'replace', 'append')
    """
    try:
        with engine.connect() as conn:
            metadata = MetaData()

            # Define table schema based on DataFrame's column types
            columns = []

            for col in df.columns:
                dtype = df[col].dtype
                
                # Map Pandas dtype to MySQL/SQLAlchemy types
                if pd.api.types.is_integer_dtype(dtype):
                    col_type = Integer
                elif pd.api.types.is_float_dtype(dtype):
                    col_type = Float
                elif pd.api.types.is_datetime64_any_dtype(dtype):
                    col_type = DateTime
                elif pd.api.types.is_bool_dtype(dtype):
                    col_type = Boolean
                elif pd.api.types.is_string_dtype(dtype):
                    col_type = Text(collation="utf8mb4_unicode_ci")  # UTF-8 support
                else:
                    col_type = String(255, collation="utf8mb4_unicode_ci")  # Default fallback

                columns.append(Column(col, col_type))

            table = Table(table_name, metadata, *columns, extend_existing=True)

            # Create the table with utf8mb4 encoding
            metadata.create_all(conn)

        # Convert object columns to string before uploading
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype(str)

        # Upload DataFrame to MySQL with proper type mapping
        dtype_mapping = {
            col: (
                sqlalchemy.types.Integer if pd.api.types.is_integer_dtype(df[col]) else
                sqlalchemy.types.Float if pd.api.types.is_float_dtype(df[col]) else
                sqlalchemy.types.DateTime if pd.api.types.is_datetime64_any_dtype(df[col]) else
                sqlalchemy.types.Boolean if pd.api.types.is_bool_dtype(df[col]) else
                sqlalchemy.types.Text(collation="utf8mb4_unicode_ci") if pd.api.types.is_string_dtype(df[col]) else
                sqlalchemy.types.String(255, collation="utf8mb4_unicode_ci")
            )
            for col in df.columns
        }

        df.to_sql(table_name, engine, if_exists=if_exists, index=False, dtype=dtype_mapping)

        print(f"✅ Data successfully uploaded to `{table_name}` with correct column types!")

    except Exception as e:
        print(f"❌ Error uploading data to `{table_name}`: {e}")


✅ Connected to MySQL successfully!


# 2. Import file plan

In [2]:
import pandas as pd
df_plan = pd.read_excel('/Users/khanhvx/Desktop/PackageChange plan.xlsx')
df_plan.head()

,Campaign,Funnel,Category,Brand,Start_date,End_date,Objective,Region,Platform,Buying_type,...,frequency_estimate,frequency_estimate_week,frequency_combine_estimate_week,er_estimate,er_bm,vtr_estimate,vtr_bm,ctr_estimate,ctr_bm,exchange_rate
0,PackageChange,AW,SBTE,Optimum,2025-03-07,2025-05-04,View Reach Campaign,NAT,YouTube,Auction,...,5.6,0.7,NaN,NaN,NaN,0.650,0.650,0.0015,0.0015,25192
1,PackageChange,AW,SBTE,Optimum,2025-03-07,2025-05-04,Reach,NAT,Facebook,Auction,...,10.6,1.2,NaN,0.001,0.001,0.005,0.005,0.0005,0.0005,25192
2,PackageChange,AW,SBTE,Optimum,2025-03-07,2025-05-04,Reach,NAT,TikTok,Auction,...,12.8,1.5,NaN,NaN,NaN,0.030,0.030,0.0005,0.0005,25192
3,PackageChange,AW,SBTE,Optimum,2025-03-07,2025-05-04,Reach,NAT,OTT,Booking,...,4.0,0.5,NaN,NaN,NaN,0.600,0.600,0.0005,0.0005,25192
4,PackageChange,AW,SBTE,Optimum Gold,2025-03-07,2025-05-04,View Reach Campaign,NAT,YouTube,Auction,...,6.1,0.7,NaN,NaN,NaN,0.650,0.650,0.0015,0.0015,25192


## 3. Get Data and xử lý

In [104]:
conn = mysql.connector.connect(
    host="10.0.0.3",
    port = '3306',
    user="dashboard_report_vinamilk",
    password="6vtRhNxa79zPsfs5",
    database="dashboard_report_vinamilk"
)

cursor = conn.cursor()



df_tt = get_data("tiktok_template_5")
df_yt = get_data("googleads_performance_daily_raw_v1")
df_fb = get_data("facebook_performance_daily_raw")

if 'cursor' in locals():
    cursor.close()
if 'conn' in locals() and conn.is_connected():
    conn.close()
    print("MySQL connection closed.")

MySQL connection closed.


In [15]:
df_map_camp = pd.read_excel('/Users/khanhvx/Downloads/map.xlsx', sheet_name='campaign')
df_map_ad = pd.read_excel('/Users/khanhvx/Downloads/map.xlsx', sheet_name='ad')
df_map_camp = df_map_camp.drop_duplicates(subset='campaign_id', keep='last')
df_map_ad = df_map_ad.drop_duplicates(subset='ad_id', keep='last')


# Merge campaign name
df_fb = df_fb.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')
df_fb['campaign_name_y'] = df_fb['campaign_name_y'].fillna(df_fb['campaign_name_x'])
df_fb = df_fb.rename(columns={'campaign_name_y': 'campaign_name'}).drop(columns=['campaign_name_x'])

# Merge ad (dd_name) from ad_id
df_fb = df_fb.merge(df_map_ad[['ad_id', 'ad_name']], on='ad_id', how='left')
df_fb['ad_name_y'] = df_fb['ad_name_y'].fillna(df_fb['ad_name_x'])
df_fb = df_fb.rename(columns={'ad_name_y': 'ad_name'}).drop(columns=['ad_name_x'])


# Merge campaign name
df_yt = df_yt.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')
df_yt['campaign_name'] = df_yt['campaign_name'].fillna(df_yt['campaign'])
df_yt = df_yt.drop(columns=['campaign'])

# Merge ad (dd_name) from ad_id
df_yt = df_yt.merge(df_map_ad[['ad_id', 'ad_name']], on='ad_id', how='left')
df_yt['ad_name'] = df_yt['ad_name'].fillna(df_yt['ad'])
df_yt = df_yt.drop(columns=['ad'])

# Merge campaign name
df_tt = df_tt.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')
df_tt['campaign_name_y'] = df_tt['campaign_name_y'].fillna(df_tt['campaign_name_x'])
df_tt = df_tt.rename(columns={'campaign_name_y': 'campaign_name'}).drop(columns=['campaign_name_x'])

# Merge ad (dd_name) from ad_id
df_tt = df_tt.merge(df_map_ad[['ad_id', 'ad_name']], on='ad_id', how='left')
df_tt['ad_name_y'] = df_tt['ad_name_y'].fillna(df_tt['ad_name_x'])
df_tt = df_tt.rename(columns={'ad_name_y': 'ad_name'}).drop(columns=['ad_name_x'])

In [16]:
df_yt['campaign_name'] = df_yt['campaign_name'].str.replace('-0304-0405', '-0703-0405')
df_fb['campaign_name'] = df_fb['campaign_name'].str.replace('-0304-0405', '-0703-0405')
df_tt['campaign_name'] = df_tt['campaign_name'].str.replace('-0304-0405', '-0703-0405')



In [17]:
df_fb = df_fb.rename(columns={'impression': 'impressions'})
df_total = pd.concat([df_yt, df_fb, df_tt], ignore_index=True)
df_total_packagechange = df_total[df_total.campaign_name.str.contains('PackageChange', na=False)]

In [6]:
import pandas as pd


# Ensure report_date is in datetime format
df_total['report_date'] = pd.to_datetime(df_total['report_date'])

# Group by campaign_name, campaign_id, and weekly bins from Monday to Sunday
df_weekly = df_total.groupby([
    pd.Grouper(key='report_date', freq='W-SUN'),
    'campaign_name',
    'campaign_id'
]).agg({'impressions': 'sum'}).reset_index()

# Adjust report_date to start of the week (Monday)
df_weekly['report_date'] = df_weekly['report_date'] - pd.Timedelta(days=6)

# Display the result
df_weekly

,report_date,campaign_name,campaign_id,impressions
0,2024-12-16,AW-SCA-Sữa Chua Ăn-Nha Đam-1804-1305_TT_REA_A...,1830097232981121,0
1,2024-12-16,CDS-SCUMS-Probi-SustainQ1-2701-2802_TT_VIEW_A...,1821946962617361,0
2,2024-12-16,(NEW) CVR-AlLCate-DTC-AWO-Livestream-0105-3105...,1831066737959985,0
3,2024-12-16,(NEW) CVR-AlLCate-DTC-AWO-Livestream-0105-3105...,1831067557751825,0
4,2024-12-16,(NEW) CVR-SBTE-AllBrand-AWO-Livestream-0105-31...,1831070154644497,0
...,...,...,...,...
9520,2025-05-19,[NO USE] AW-SCA-Sữa Chua Ăn-ThematicLaunch-240...,1827557621235713,0
9521,2025-05-19,[Test] App-Engagement_ADR,22381899504,0
9522,2025-05-19,[Test] App-Engagement_IOS,22381886547,0
9523,2025-05-19,test,22082475081,0


In [7]:
df_reach= pd.read_excel('/Users/khanhvx/Downloads/reach SN.xlsx', sheet_name='Sheet1')
df_reach = df_reach.merge(
    df_weekly[['campaign_id', 'report_date', 'impressions']], 
    on=['campaign_id', 'report_date'], 
    how='left'
)

In [8]:
df_reach

,customer id,customer,campaign_name,campaign_id,Reach,impression,report_date,impressions
0,4405135390,VNM_SN_2025,AW-SN-FM100-FM100PromoSustainJan-0301-2101_YT_...,22092016942,0,0,2025-05-12,0.0
1,4405135390,VNM_SN_2025,AW-SN-FM100-FM100PromoSustainFeb-0802-2502_YT_...,22209786637,0,0,2025-05-12,0.0
2,4405135390,VNM_SN_2025,Video Sequence - 2025-03-05,22312630450,0,0,2025-05-12,0.0
3,4405135390,VNM_SN_2025,AW-SN-Fino-FinoPromoSustain-1203-3103_YT_VRC_A...,22325801436,0,0,2025-05-12,0.0
4,4405135390,VNM_SN_2025,AW-SN-Fino-FinoPromoSustain-1203-3103_YT_VRC_A...,22334929651,0,0,2025-05-12,0.0
...,...,...,...,...,...,...,...,...
676,7007534786,VNM_SN_FM100_2025_VND,AW-SN-Fino-ADMMVLaunch-1405-2505_GG_VRC_AUC_MI...,22546340285,0,0,2025-01-06,NaN
677,7007534786,VNM_SN_FM100_2025_VND,AW-SN-Fino-ADMMVLaunch-1205-2505_GG_VRC_AUC_MI...,22547514953,0,0,2025-01-06,NaN
678,7007534786,VNM_SN_FM100_2025_VND,AW-SN-Fino-ADMMVLaunch-1405-2505_GG_VRC_AUC_MI...,22546339583,0,0,2024-12-30,NaN
679,7007534786,VNM_SN_FM100_2025_VND,AW-SN-Fino-ADMMVLaunch-1405-2505_GG_VRC_AUC_MI...,22546340285,0,0,2024-12-30,NaN


In [18]:
# Assign column names safely (only for expected number of columns)
campaign_name_expected_columns = ['camp_code', 'platform', 'camp_type', 'buying_type', 'free_text1', 'year', 'optimization', 
                    'buying_model', 'placement', 'freetext_2']

adgroup_name_expected_columns = ['noneed_1', 'noneed_2', 'noneed_3', 'noneed_4', 'noneed_5', 'noneed_6', 'noneed_7', 
                    'noneed_8', 'bidding_strategy', 'noneed_9']
ad_name_expected_columns = ['creative_type', 'creative_format', 'creative_length', 'noneed_9', 'noneed_10', 'noneed_11', 'noneed_12', 
                    'noneed_13', 'freetext_3']

def split_and_assign(df, column, expected_columns):
    # Ensure column exists and is a string
    if column in df.columns:
        df[column] = df[column].fillna("").astype(str)

        # Split column
        # Split column
        split_cols = df[column].str.split('_', expand=True)

        # Get actual and expected counts
        num_actual_cols = split_cols.shape[1]
        num_expected_cols = len(expected_columns)

        # ✅ Pad with NaNs if there are fewer actual columns than expected
        if num_actual_cols < num_expected_cols:
            for i in range(num_actual_cols, num_expected_cols):
                split_cols[i] = pd.NA  # You can also use np.nan

        # ✅ Recalculate in case padding added columns
        num_actual_cols = split_cols.shape[1]

        # Assign column names
        assigned_names = expected_columns[:num_actual_cols]
        extra_names = [f"extra_{i}" for i in range(num_actual_cols - len(assigned_names))]
        all_new_names = assigned_names + extra_names
        all_new_names = [f"{name}_split" if name in df.columns else name for name in all_new_names]

        split_cols.columns = all_new_names

        # Join back to original DataFrame
        df = df.join(split_cols)

    # ✅ Fix for 'camp_code' column splitting
    if "camp_code" in df.columns:
        df["camp_code"] = df["camp_code"].fillna("").astype(str)
        campaign_split = df["camp_code"].str.split("-", expand=True)

        # ✅ Force exactly 6 columns
        max_cols = 6
        campaign_split = campaign_split.iloc[:, :max_cols]  # Trim extra columns
        for i in range(campaign_split.shape[1], max_cols):
            campaign_split[i] = np.nan  # Fill missing columns

        campaign_split.columns = ["Funnel", "Category", "Brand Name", "Campaign Name", "Start Date", "End Date"]
        
        # ✅ Drop existing columns before joining
        df = df.drop(columns=[col for col in campaign_split.columns if col in df.columns], errors="ignore")
        df = df.join(campaign_split)

    # ✅ Fix for 'free_text1' column splitting
    if "free_text1" in df.columns:
        df["free_text1"] = df["free_text1"].fillna("").astype(str)
        freetext_split = df["free_text1"].str.split("-", expand=True)

        # ✅ Force exactly 6 columns
        freetext_split = freetext_split.iloc[:, :max_cols]  # Trim extra columns
        for i in range(freetext_split.shape[1], max_cols):
            freetext_split[i] = np.nan  # Fill missing columns

        freetext_split.columns = ["Audience", "Gender", "Age", "Region", "Device", "Os"]

        # ✅ Drop existing columns before joining
        df = df.drop(columns=[col for col in freetext_split.columns if col in df.columns], errors="ignore")
        df = df.join(freetext_split)

    # ✅ Create 'TA' column (Target Audience)
# ✅ Create 'TA' column (Target Audience)
    if "Gender" in df.columns and "Age" in df.columns:
        df["Gender"] = df["Gender"].astype(str)  # Ensure Gender is string
        df["Age"] = df["Age"].astype(str).str.zfill(4)  # Ensure Age is string and has at least 4 characters
        df["TA"] = df["Gender"] + df["Age"].str[:2] + "-" + df["Age"].str[2:]

    # ✅ Convert 'Start Date' and 'End Date' into proper format
    if "Start Date" in df.columns and "End Date" in df.columns and "year" in df.columns:
        df["year"] = df["year"].astype(str)  # Ensure year is a string

        def format_date(row, col):
            if pd.notna(row[col]) and len(row[col]) == 4:
                return f"{row['year']}-{row[col][2:]}-{row[col][:2]}"
            return None

        df["Start Date"] = df.apply(lambda row: format_date(row, "Start Date"), axis=1)
        df["End Date"] = df.apply(lambda row: format_date(row, "End Date"), axis=1)

        # Convert to YYYY-MM-DD format
        df["Start Date"] = pd.to_datetime(df["Start Date"], format="%Y-%m-%d", errors="coerce")
        df["End Date"] = pd.to_datetime(df["End Date"], format="%Y-%m-%d", errors="coerce")

    if all(col in df.columns for col in ["creative_type", "creative_format", "creative_length"]):
        df["ad_format"] = df["creative_type"].astype(str) + "_" + df["creative_format"].astype(str) + "_" + df["creative_length"].astype(str)
        
    # ✅ Remove columns containing "noneed"
    df = df.drop(columns=[col for col in df.columns if "noneed" in col], errors="ignore")


    return df


def process_campaign_data(df):
    """Processes a DataFrame by filtering campaigns, splitting columns, and formatting TA & dates."""
    
    # ✅ Filter DataFrame based on campaign prefix
    df_filtered = df[df['campaign_name'].str.startswith(('AW', 'CSD', 'CVR'), na=False)].copy()

    # ✅ Apply splitting logic
    df_filtered = split_and_assign(df_filtered, 'campaign_name', campaign_name_expected_columns)
    df_filtered = split_and_assign(df_filtered, 'ad_name', ad_name_expected_columns)

    return df_filtered

df_yt_filter = process_campaign_data(df_yt)
df_fb_filter = process_campaign_data(df_fb)
df_tt_filter = process_campaign_data(df_tt)

In [10]:
def mapping_naming(df):
    mapping_creative_type = {
        "VID": "Video",
        "IMAGE": "Image",
        "BAN": "Banner",
        "AUD": "Audio",
        "TEXT": "Text",
        "DISC": "Discovery Ad",
        "RESAD": "Responsive Ad",
        "ST": "Story",
        "GIF": "Gif",
        "CARO": "Carousel",
        "REEL": "Reel",
        "MULT": "Multiple",
        "DNM": "dynamic",
        "CLT": "collec",
        "TVC": "tvc",
        "SMS": "SMS",
        "LIVESTREAM": "livetream"
    }
    mapping_campaign_type = {
        "VRC": "View Reach Campaign",
        "VVC": "Video View Campaign",
        "PMAX": "PMAX",
        "DG": "DemandGen",
        "SEM": "SEM",
        "VAC": "Video Action Campaign",
        "REA": "Reach",
        "ENG": "Engage",
        "TRF": "Traffic",
        "LEAD": "Lead",
        "THRU": "Thruplay",
        "PDIS": "Programatic Display",
        "PVID": "Programatic Video",
        "OTT": "Programatic OTT",
        "MH": "Masthead",
        "App": "App",
        "GDN": "GDN",
        "IS": "InStream",
        "BUMP": "Bumper",
        "TVN": "Trueview Nonskip",
        "LSA": "Livestream Ads",
        "CT": "Community Interaction",
        "PSA": "Product Shopping Ads",
        "VSA": "Video Shopping Ads"
    }
    mapping_platform = {
    "DV360": "DV360",
    "DV360-DISP": "DV360 Display",
    "DV360-VID": "DV360 Video (non-YouTube)",
    "DV360-YT": "DV360 YouTube",
    "GG": "Google Ads",
    "GA-YT": "Google Ads YouTube",
    "GDN": "Google Display Network",
    "D-YT": "YouTube Direct",
    "FB": "Facebook",
    "TT": "TikTok",
    "IG": "Instagram"
    }
    mapping_campaign_funnel = {
    "AW": "Awareness",
    "CSD": "Consideration",
    "CVR": "Conversion"
    }

    # Convert creative_type first
    df["creative_type"] = df["creative_type"].map(mapping_creative_type)
    df["camp_type"] = df["camp_type"].map(mapping_campaign_type)
    df["platform"] = df["platform"].map(mapping_platform)
    df["Funnel"] = df["Funnel"].map(mapping_campaign_funnel)


    # Create ad_format_plan column

mapping_naming(df_yt_filter)
mapping_naming(df_fb_filter)
mapping_naming(df_tt_filter)
df_fb_filter["ad_format_plan"] = df_fb_filter["creative_type"].astype(str) + " " + df_fb_filter["creative_length"].astype(str)

### check campaign status

In [12]:

import pandas as pd
from datetime import timedelta

def detect_inactive_campaigns_by_date(df):
    """
    Detects inactive campaigns for each unique report_date in the dataset.
    A campaign is considered 'Done' on a given report_date if it had
    zero cost, impressions, and clicks in the 7 days prior to that date.
    
    Returns a DataFrame with 'check_date', 'campaign_id', and 'Status'.
    """
    df = df.copy()
    df['report_date'] = pd.to_datetime(df['report_date'])
    
    results = []
    all_campaigns = df['campaign_id'].unique()

    for check_date in sorted(df['report_date'].unique()):
        # Define time window: 7 days before check_date
        start_date = check_date - timedelta(days=7)
        recent = df[(df['report_date'] >= start_date) & (df['report_date'] < check_date)]
        
        # Sum metrics by campaign
        sums = (recent
                .groupby('campaign_id')[['cost','impressions','clicks']]
                .sum()
                .reset_index())

        # Include all campaigns
        merged = pd.DataFrame({'campaign_id': all_campaigns})
        merged = (merged
                  .merge(sums, on='campaign_id', how='left')
                  .fillna(0))

        # Determine status
        merged['Status'] = merged.apply(
            lambda row: 'Done' if (row[['cost','impressions','clicks']] == 0).all() 
                        else 'On Going',
            axis=1
        )

        merged['check_date'] = check_date.strftime('%Y-%m-%d')
        results.append(merged[['check_date', 'campaign_id', 'Status']])
    
    return pd.concat(results, ignore_index=True)


df_total_filter = pd.concat([df_yt_filter, df_fb_filter, df_tt_filter], ignore_index=True)
inactive_campaigns = detect_inactive_campaigns_by_date(df_total_filter)
inactive_campaigns

,check_date,campaign_id,Status
0,2024-12-19,22209786637,Done
1,2024-12-19,22092016942,Done
2,2024-12-19,22477269516,Done
3,2024-12-19,22449604773,Done
4,2024-12-19,22356365700,Done
...,...,...,...
77211,2025-05-19,1821474746296385,Done
77212,2025-05-19,1825576423191585,Done
77213,2025-05-19,1832177492056097,Done
77214,2025-05-19,1832250978302001,On Going


### Check lasted day

In [25]:
import pandas as pd

def get_latest_active_date_per_campaign(df):
    # Ensure report_date is datetime
    df['report_date'] = pd.to_datetime(df['report_date'])

    # Filter rows with non-zero metrics
    active_df = df[(df['cost'] != 0) & 
                   (df['impressions'] != 0) & 
                   (df['clicks'] != 0)]

    # Group by camp_code and get the latest active date
    result_df = (active_df.groupby('camp_code')['report_date']
                          .max()
                          .reset_index()
                          .rename(columns={'report_date': 'latest_active_date'}))
    
    return result_df


latest_active_df = get_latest_active_date_per_campaign(df_total_filter)
latest_active_df

,camp_code,latest_active_date
0,AW,2025-05-15
1,AW-?NTC-NTC-SocialAWOFeb-1102-2102,2025-03-14
2,AW-?NTC-Vfresh-SocialAWOJan-0901-1901,2025-02-03
3,AW-?SCUMS-Probi-SocialAWOJan-0801-1801,2025-02-13
4,AW-?SN-FM100-SocialAWOFeb-2402-0503,2025-03-19
...,...,...
169,CVR-SN-GreenFarm-Livestream-2002-2002,2025-02-20
170,CVR-SN-GreenFarm-Livestream-2302-2302,2025-02-23
171,CVR-SN-GreenFarm-Livestream-2502-2502,2025-02-25
172,CVR-SN-GreenFarm-Livestream-2702-2702,2025-02-27


In [ ]:
# Establish MySQL connection
DB_TYPE = 'mysql'  # Change to 'postgresql' for PostgreSQL
DB_HOST = '125.212.245.36'
DB_PORT = '3306'  # Change for different databases
DB_USER = 'dashboard_report_vinamilk'
DB_PASS = '6vtRhNxa79zPsfs5'
DB_NAME = 'dashboard_report_vinamilk'


# Connect to MySQL
try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        print("✅ Connected to MySQL successfully!")
    else:
        print("❌ Failed to connect.")

except mysql.connector.Error as e:
    print(f"Error: {e}")
# Create SQLAlchemy engine
cursor = conn.cursor()
cursor.execute("SET NAMES utf8mb4;") 
cursor.execute("SET CHARACTER SET utf8mb4;")
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
upload_to_mysql(inactive_campaigns, 'mapping_campaign_status', engine)
cursor.close()
conn.close()

✅ Connected to MySQL successfully!
❌ Error uploading data to `mapping_campaign_status`: (pymysql.err.DataError) (1264, "Out of range value for column 'campaign_id' at row 1")
[SQL: INSERT INTO mapping_campaign_status (check_date, campaign_id, `Status`) VALUES (%(check_date)s, %(campaign_id)s, %(Status)s)]
[parameters: [{'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22209786637, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22092016942, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22477269516, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22449604773, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22356365700, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 22325801436, 'Status': 'Done'}, {'check_date': datetime.datetime(2024, 12, 19, 0, 0), 'campaign_id': 2233

In [ ]:
import pandas as pd

def process_dataframe(df, column_mapping, numeric_cols, extra_processing=None, column_order=None):
    """General function to process dataframes."""
    # Select and rename columns
    existing_columns = [col for col in column_mapping if col in df.columns]
    df = df[existing_columns].rename(columns=column_mapping).copy()
    
    # Convert 'Date' column
    df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce')
    df['report_date'] = pd.to_datetime(df['report_date'], errors='coerce').dt.date

    # Convert numeric columns
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Apply dataset-specific processing
    if extra_processing:
        extra_processing(df)
    
    # Ensure all expected columns exist
    if column_order:
        for col in column_order:
            if col not in df.columns:
                df[col] = None
        df = df[column_order]
    
    return df


# Define column mappings & numeric columns for each dataset
youtube_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'buying_type': 'Buying_Type',
    'TA': 'TA',
    'Region': 'Region',
    'campaign_name': 'Campaign_Name',
    'campaign_id': 'Campaign_Id',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'camp_type': 'Format',
    'impressions': 'Impression',
    'clicks': 'Clicks',
    'views': 'Views',
    'currency_code': 'Currency_Code',
    'cost': 'Cost_USD',
    'engagements': 'Engagements',
    'video_25': 'Video_Plays_25',
    'video_50': 'Video_Plays_50',
    'video_75': 'Video_Plays_75',
    'video_100': 'Video_Plays_100'}  
tiktok_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'TA': 'TA',
    'Region': 'Region',
    'camp_type': 'Format',
    'buying_type': 'Buying_Type',
    'campaign_name': 'Campaign_Name',
    'campaign_id': 'Campaign_Id',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'spend': 'Cost',
    'impressions': 'Impression',
    'clicks_destination': 'Clicks',
    'engagements': 'Engagements',
    'profile_visits': 'profile_visits',
    '2_second_video_views': '23s_Video_Views',
    '6_second_video_views': 'Views',
    'engaged_view': '6s_Engaged_Views',
    'video_views_p25': 'Video_Plays_25',
    'video_views_p50': 'Video_Plays_50',
    'video_views_p75': 'Video_Plays_75',
    'video_views_p100': 'Video_Plays_100'}
facebook_mapping = {'report_date': 'report_date',
    'Category': 'Category',
    'platform': 'Platform',
    'Brand Name': 'Brand',
    'camp_type': 'Format',
    'buying_type': 'Buying_Type',
    'TA': 'TA',
    'Region': 'Region',
    'campaign_name': 'Campaign_Name',
    'campaign_id': 'Campaign_Id',
    'camp_code': 'Campaign_code',
    'Campaign Name': 'Campaign',
    'Start Date': 'Start_Date',
    'End Date': 'End_Date',
    'freetext_2': 'freetext_2',
    'impressions': 'Impression',
    'cost': 'Cost',
    'post_engagement': 'Engagements',
    'video_played_3': '23s_Video_Views',
    'thruplays': 'Views',
    'video_played_25': 'Video_Plays_25',
    'video_played_50': 'Video_Plays_50',
    'video_played_75': 'Video_Plays_75',
    'video_played_complete': 'Video_Plays_100',
    'link_click': 'Clicks'}

youtube_numeric_cols = ['Impression', 'Clicks', 'Views', 'Cost', 'Engagements', 
    'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100']
tiktok_numeric_cols = ['Cost', 'Impression', 'Clicks', 'Engagement',
    '2s_Video_Views', '6s_Video_Views', '6s_Engaged_Views',
   'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100']
facebook_numeric_cols = ['Engagement', '3s_Video_Plays', 'Impression',
    'Cost', 'Clicks', 'Video_Played_25', 'Video_Played_50', 'Video_Played_75', 
    'Video_Played_100', 'Thruplays']


# Process datasets
report_campaign_overall_youtube = process_dataframe(df_yt_filter, youtube_mapping, youtube_numeric_cols)
report_campaign_overall_tiktok = process_dataframe(df_tt_filter, tiktok_mapping, tiktok_numeric_cols)
report_campaign_overall_facebook = process_dataframe(df_fb_filter, facebook_mapping, facebook_numeric_cols)

In [ ]:
report_campaign_overall_youtube['Platform'] = 'YouTube'
report_campaign_overall_tiktok['Platform'] = 'TikTok'
report_campaign_overall_facebook['Platform'] = 'Facebook'

In [ ]:
report_campaign_overall_total = pd.concat([report_campaign_overall_youtube, report_campaign_overall_tiktok, report_campaign_overall_facebook], ignore_index=True)

In [ ]:
report_campaign_overall_total['freetext_2'] = report_campaign_overall_total['freetext_2'].replace('Nhom1 adhoc', 'Nhom1')
report_campaign_overall_total['freetext_2'] = report_campaign_overall_total['freetext_2'].replace('G34', 'G3G4')


In [ ]:
df_plan['Brand'] = df_plan['Brand'].replace('Optimum Master', 'Optimum')
df_plan['Brand'] = df_plan['Brand'].replace('Dielac Gold', 'DG')
df_plan['Brand'] = df_plan['Brand'].replace('Dielac Grow Plus', 'DGP')
df_plan = df_plan.rename(columns={"Freetext1": "freetext_2"})

In [ ]:
# Ensure Start Date and End Date are datetime
df_plan["Start Date"] = pd.to_datetime(df_plan["Start Date"], errors="coerce")
df_plan["End Date"] = pd.to_datetime(df_plan["End Date"], errors="coerce")

In [ ]:
df_plan.columns = df_plan.columns.str.replace(' ', '_')

In [ ]:
# Assuming df_plan has a column 'exchange_rate'
exchange_rate_value = df_plan['exchange_rate'].iloc[0]  # Get the value from the first row

# Now you can assign this value to the 'report_campaign_overall_total' DataFrame
report_campaign_overall_total['exchange_rate'] = exchange_rate_value

# Function to calculate cost based on platform and currency_code
def calculate_cost(row):
    if row['Platform'] == 'YouTube':
        if row['Currency_Code'] == 'USD':
            return row['Cost_USD'] * row['exchange_rate']
        elif row['Currency_Code'] == 'VND':
            return row['Cost_USD'] * 1  # No exchange rate needed for VND
        else:
            return row['Cost_USD']  # For other currencies, just return the cost in USD
    else:
        # No changes or calculations for other platforms, return the cost as is
        return row['Cost']

# Apply the function to each row in the DataFrame
report_campaign_overall_total['Cost'] = report_campaign_overall_total.apply(calculate_cost, axis=1)

In [ ]:
def preprocess_campaign_data(df):
    # Rename column
    df = df.rename(columns={'CATE': 'Category'})
    
    # Define columns
    numeric_cols = ['Impression', 'Clicks', 'Views', 'Cost', 'Engagements', 'Video_Plays_25',
              'Video_Plays_50', 'Video_Plays_75', 'Video_Plays_100', '23s_Video_Views', '6s_Engaged_Views', 'profile_visits']
    
    group_cols = ['Category', 'Brand', 'Campaign', 'Platform', 'freetext_2', 'Start_Date', 'End_Date', 'Campaign_code', 'Format', 'TA']

    # Group and sum
    groupdf = df.groupby(group_cols, dropna=False)[numeric_cols].sum().reset_index()
    
    # Convert dates
    groupdf['Start_Date'] = pd.to_datetime(groupdf['Start_Date'], errors='coerce')
    groupdf['End_Date'] = pd.to_datetime(groupdf['End_Date'], errors='coerce')
    
    # Campaign duration
    groupdf['campaign_duration'] = (groupdf['End_Date'] - groupdf['Start_Date']).dt.days

    # Set report_date as max End_Date from original df
    report_date = pd.to_datetime(df['report_date'], errors='coerce').max().normalize()
    groupdf['report_date'] = report_date

    # Active day = from Start_Date to report_date
    groupdf['active_day'] = (groupdf['report_date'] - groupdf['Start_Date']).dt.days
    
    return groupdf

groupdf = preprocess_campaign_data(report_campaign_overall_total)

In [ ]:
print("--- Initial Data Check ---")
# Convert to datetime and handle errors
report_campaign_overall_total['report_date'] = pd.to_datetime(report_campaign_overall_total['report_date'], errors='coerce')

# Drop rows where date conversion failed
initial_rows = len(report_campaign_overall_total)
report_campaign_overall_total = report_campaign_overall_total.dropna(subset=['report_date'])
print(f"Dropped {initial_rows - len(report_campaign_overall_total)} rows due to invalid dates.")

# Check date range of the source DataFrame
if not report_campaign_overall_total.empty:
    min_date = report_campaign_overall_total['report_date'].min()
    max_date = report_campaign_overall_total['report_date'].max()
    print(f"Source data date range: {min_date} to {max_date}")
    print(f"Source columns: {report_campaign_overall_total.columns.tolist()}") # Show all columns
else:
    print("Source DataFrame is empty after handling invalid dates.")
    # exit()


# --- Processing Loop for Cumulative Data ---
print("\n--- Processing Loop (Cumulative) ---")
start_date = datetime(2025, 4, 1)
end_date = datetime(2025, 4, 24)

# Use a dictionary to store the results
daily_cumulative_reports = {}

if not report_campaign_overall_total.empty:
    for i in range((end_date - start_date).days + 1):
        day = start_date + timedelta(days=i)
        day_key = f"day_{i + 1}" # e.g., day_1, day_2

        # Normalize the date column for comparison (removes time part)
        normalized_dates = report_campaign_overall_total['report_date'].dt.normalize()

        # Filter rows where report_date <= current day (cumulative)
        # Use pd.Timestamp for comparison robustness
        current_day_timestamp = pd.Timestamp(day).normalize()

        # *** This filter selects rows based on date BUT keeps ALL original columns ***
        df_day_cumulative = report_campaign_overall_total[normalized_dates <= current_day_timestamp].copy()
        # Using .copy() is good practice here to ensure the slice is independent

        # Store the resulting DataFrame (with all its columns) in the dictionary
        daily_cumulative_reports[day_key] = df_day_cumulative

        # Debug print: show the day being processed and the shape of the resulting df
        print(f"Processed {day.strftime('%Y-%m-%d')} ({day_key}): Found {len(df_day_cumulative)} cumulative rows.")

else:
    print("Skipping loop because source DataFrame has no valid date entries.")


In [ ]:
from datetime import datetime, timedelta
report_campaign_overall_total['report_date'] = pd.to_datetime(report_campaign_overall_total['report_date'], errors='coerce')

# Create one DataFrame per day from 2025-04-01 to 2025-04-10
start_date = datetime(2025, 4, 1)
end_date = datetime(2025, 4, 24)

for i in range((end_date - start_date).days + 1):
    day = start_date + timedelta(days=i)
    day_str = str(i + 1)  # 1, 2, ..., 10

    # Filter rows where report_date == current day
    df_day = report_campaign_overall_total[
        report_campaign_overall_total['report_date'].dt.normalize() <= pd.to_datetime(day)
    ]

    # Assign to dynamic variable
    globals()[f'report_campaign_overall_total_{day_str}'] = df_day


In [ ]:
grouped_cumulative_reports = {}  # To store processed DataFrames

for day_key, df in daily_cumulative_reports.items():
    try:
        groupdf = preprocess_campaign_data(df)

        if groupdf is None:
            print(f"[Warning] preprocess_campaign_data() returned None for {day_key}")
        else:
            grouped_cumulative_reports[day_key] = groupdf
            print(f"[Info] Successfully processed {day_key}")
            
    except Exception as e:
        print(f"[Error] Failed to process {day_key}: {e}")

for i in range(1, 21):
    key = f'day_{i}'
    if key in grouped_cumulative_reports:
        globals()[f'grouped_df_day{i}'] = grouped_cumulative_reports[key]


In [ ]:
# Combine all DataFrames and add a 'day' column to keep track
groupdf = pd.concat(
    [df.assign(day=key) for key, df in grouped_cumulative_reports.items()],
    ignore_index=True
)

In [ ]:
df_ga= pd.read_excel('/Users/khanhvx/Downloads/df_ga.xlsx')
# Step 1: Replace 'Toàn Quốc' with 'NAT' in Geo
df_ga['Geo'] = df_ga['Geo'].replace('Toàn Quốc', 'NAT')

# Step 2: Create TA column from Gender and Age
def make_ta(gender, age):
    try:
        first_letter = gender[0].upper()
        age_range = f"{age[:2]}-{age[2:]}"
        return f"{first_letter}{age_range}"
    except:
        return None  # or you can return f"{gender}_{age}" as fallback

df_ga['TA'] = df_ga.apply(lambda row: make_ta(row['Gender'], row['Age']), axis=1)
df_ga
df_ga_summary = df_ga.groupby([
    'Campaign Name', 
    'Cate', 
    'Brand', 
    'Start Date', 
    'Buying Method', 
    'Geo', 
    'Platform', 
    'TA'
])[[
    'sessions', 
    'engagedSessions', 
    'addToCarts', 
    'purchaseRevenue', 
    'ecommercePurchases', 
    'screenPageViews'
]].sum().reset_index()

df_ga_summary = df_ga_summary.rename(columns={
    'Campaign Name': 'Campaign',
    'Cate': 'Category',
    'Start Date': 'Start_Date',
    'Buying Method': 'Buying_Method',
    'Geo': 'Region'
})
df_ga_summary


In [27]:
import pandas as pd

# Assuming df_ga_summary and report_campaign_summary are already loaded

merge_cols = ['Campaign', 'Category', 'Platform', 'Brand', 'Start_Date']

# --- Refined Normalization ---
def normalize_cols(df, cols, prefix):
    for col in cols:
        # Handle dates explicitly first if they are datetime objects
        if pd.api.types.is_datetime64_any_dtype(df[col]):
             # Normalize to 'YYYY-MM-DD' format. Adjust if needed.
            df[f'{prefix}_{col}'] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')
        else:
            # Apply string normalization for others
            df[f'{prefix}_{col}'] = df[col].astype(str).str.strip().str.lower().str.replace(r'\s+', '', regex=True)
            # Handle potential 'nan' strings if they should be treated as actual NaN for merging (optional)
            # df[f'{prefix}_{col}'] = df[f'{prefix}_{col}'].replace('nan', pd.NA)
    return df

# Apply normalization
df_ga_summary = normalize_cols(df_ga_summary.copy(), merge_cols, 'ga') # Use .copy() to avoid SettingWithCopyWarning
report_campaign_summary = normalize_cols(groupdf.copy(), merge_cols, 'total')

# Step 1: Normalize (already done)

# Step 2: Group and sum GA summary
ga_grouped = df_ga_summary.groupby(
    [f'ga_{col}' for col in merge_cols], as_index=False
).sum(numeric_only=True)  # Ensures only numeric metric columns are summed

# Step 3: Merge the summed GA data with the report summary
merged_df_ga = pd.merge(
    report_campaign_summary,
    ga_grouped,
    how='left',
    left_on=[f'total_{col}' for col in merge_cols],
    right_on=[f'ga_{col}' for col in merge_cols],
    suffixes=('', '_ga')
)

# Step 4: Drop helper columns
merged_df_ga = merged_df_ga.drop(
    columns=[col for col in merged_df_ga.columns if col.startswith('ga_') or col.startswith('total_') or col.endswith('_total')]
)

merged_df_ga

NameError: name 'df_ga_summary' is not defined

In [ ]:
# Create lowercase versions of the key columns for both dataframes
merge_keys =  ['Campaign', 'Funnel', 'Category', 'Region', 'Platform', 'Brand', 'TA', 'Buying_Method', 'Plan_Start_Date', 'Actual_Start_Date', 'Audience_group']

# Merge using the lowercase versions
merged_df = pd.merge(
    merged_df_ga,
    df_plan,
    on=merge_keys,
    how='inner')


NameError: name 'merged_df_ga' is not defined

In [ ]:
merged_df

In [ ]:
merged_df.drop_duplicates()

In [ ]:
merged_df

In [ ]:
(merged_df['Brand'] + ' - ' + merged_df['freetext_2']).value_counts().sort_index()

In [ ]:
total_cost = merged_df[merged_df['report_date'] == '2025-04-20 00:00:00']['Net_media_cost'].sum()
print(f"Total Net Media Cost on 2025-04-20: {total_cost}")

In [ ]:
df_total_packagechange = df_total[df_total.campaign_name.str.contains('PackageChange')]
df_total_packagechange.cost.sum()

In [ ]:
# Assuming each campaign_name maps to a unique campaign_id
df_campaigns = df_total_packagechange[['campaign_name', 'campaign_id']].drop_duplicates().reset_index(drop=True)
df_campaigns

In [ ]:
df_total_packagechange


In [ ]:
merged_df

In [ ]:
merged_df['Funnel'] = merged_df['Funnel'].replace('AW', 'Awareness')

In [ ]:
merged_df

In [ ]:
# Ensure report_date is in datetime format

# Get the latest date in the dataset
max_report_date = merged_df['report_date'].max()

# Create the Lasted Day column
merged_df['Lasted_Day'] = merged_df['report_date'] == max_report_date

merged_df

In [ ]:
merged_df['Audience'] = merged_df['freetext_2'].astype(str) + '\n' + merged_df['TA_x'].astype(str)


# Ensure datetime types
merged_df['report_date'] = pd.to_datetime(merged_df['report_date'])
merged_df['Start_Date_x'] = pd.to_datetime(merged_df['Start_Date_x'])

# Calculate custom week number from Start_Date_x, starting from Monday
merged_df['weeknum'] = ((merged_df['report_date'] - merged_df['Start_Date_x']).dt.days // 7) + 1

# Format the new date column
merged_df['formatted_date'] = merged_df['report_date'].dt.strftime('%b %d, %Y') + \
                               ' - ' + merged_df['report_date'].dt.day_name() + \
                               ' - Week ' + merged_df['weeknum'].astype(str)


merged_df

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

DB_TYPE = 'mysql'  # Change to 'postgresql' for PostgreSQL
DB_HOST = '10.0.0.3'
DB_PORT = '3306'  # Change for different databases
DB_USER = 'dashboard_report_vinamilk'
DB_PASS = '6vtRhNxa79zPsfs5'
DB_NAME = 'dashboard_report_vinamilk'

try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        print("Connected to MySQL successfully!")
    else:
        print("Failed to connect.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

# Create SQLAlchemy engine
cursor = conn.cursor()
cursor.execute("SET NAMES utf8mb4;") 
cursor.execute("SET CHARACTER SET utf8mb4;")
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

def upload_to_mysql(df, table_name, engine, if_exists='replace'):
    """
    Uploads a DataFrame to a MySQL table.
    
    :param df: Pandas DataFrame to upload
    :param table_name: Name of the table in the database
    :param engine: SQLAlchemy engine object
    :param if_exists: What to do if the table already exists ('fail', 'replace', 'append')
    """
    try:
        # Upload DataFrame to MySQL
        df.to_sql(table_name, engine, if_exists=if_exists, index=False)
        
        print(f"Data successfully uploaded to {table_name}!")
    except Exception as e:
        print(f"Error uploading data to {table_name}: {e}")

# Example usage:
# engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
# upload_to_mysql(df_tt, 'report_campaign_digital_tiktok', engine)

In [ ]:
merged_df.Cost.sum()

In [ ]:
merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('-0304-0405', '-0703-0405')
merged_df['Campaign_code'].value_counts()

In [ ]:
upload_to_mysql(merged_df, 'report_campaign_overall_total', engine)

In [ ]:
upload_to_mysql(df_plan, 'report_campaign_overall_mediaplan', engine)

In [ ]:
merged_df.Cost.sum()

In [ ]:

merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('Optimum Gold', 'Optimum')
merged_df['Campaign_code'] = merged_df['Campaign_code'].str.replace('Optimum Colos', 'Optimum')


In [ ]:
merged_df.to_excel('/Users/khanhvx/Downloads/test.xlsx', index=False)

In [ ]:
merged_df2= merged_df.copy()

In [ ]:


try:
    # Establish MySQL connection
    conn = mysql.connector.connect(
        host="10.0.0.3",
        port = '3306',
        user="dashboard_report_vinamilk",
        password="6vtRhNxa79zPsfs5",
        database="dashboard_report_vinamilk"
    )

    print("Connected to MySQL successfully!")

    # Create a cursor and execute the query
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM report_campaign_overall_total")

    # Fetch column names
    column_names = [desc[0] for desc in cursor.description]

    # Fetch all rows and convert to DataFrame
    rows = cursor.fetchall()
    df_check2 = pd.DataFrame(rows, columns=column_names)
    

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("MySQL connection closed.")

df_check2.head()

In [ ]:
import mysql.connector
try:
    # Connect to MySQL
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        print("✅ Connected to MySQL successfully!")

        # Drop tables
        tables_to_drop = ["report_campaign_overall_total"]
        
        for table in tables_to_drop:
            try:
                cursor.execute(f"DROP TABLE IF EXISTS {table};")
                print(f"✅ Table {table} dropped successfully!")
            except mysql.connector.Error as e:
                print(f"❌ Error dropping table {table}: {e}")

        # Commit changes
        conn.commit()

except mysql.connector.Error as e:
    print(f"❌ Error: {e}")

finally:
    # Close connection
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("✅ MySQL connection closed.")

In [ ]:
conn = mysql.connector.connect(
    host="10.0.0.3",
    port = '3306',
    user="dashboard_report_vinamilk",
    password="6vtRhNxa79zPsfs5",
    database="dashboard_report_vinamilk"
)

cursor = conn.cursor()



merge_df = get_data("report_campaign_overall_total")

if 'cursor' in locals():
    cursor.close()
if 'conn' in locals() and conn.is_connected():
    conn.close()
    print("MySQL connection closed.")

In [ ]:
reach_df = pd.read_excel('/Users/khanhvx/Downloads/SBTE Package Change_Reach weekly.xlsx', sheet_name='Reach')
merged_df1 = merge_df.copy()
merged_df1 = merged_df1.drop(columns='Reach', errors='ignore')
merged_df1 = pd.merge(merged_df1, reach_df, how='left', on=['Campaign_code', 'Platform', 'Brand', 'freetext_2'])

In [ ]:
merged_df1

In [ ]:
# Align Start_Date_x to the most recent Sunday (week starts on Sunday)
start_week_monday = merged_df1['Start_Date_x'] - pd.to_timedelta(merged_df1['Start_Date_x'].dt.weekday, unit='D')

# Calculate week number based on Sunday-starting weeks
merged_df1['weeknum'] = ((merged_df1['report_date'] - start_week_monday).dt.days // 7) + 1

# Format date with short weekday
merged_df1['formatted_date'] = merged_df1['report_date'].dt.strftime('%b %d, %Y') + \
                              ' - ' + merged_df1['report_date'].dt.strftime('%a') + \
                              ' - Week ' + merged_df1['weeknum'].astype(str)